In [1]:
from pathlib import Path
import numpy as np
import meshcat
import pickle
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import copy
import time

from rdt.common import path_util, util
from rdt.camera.simple_multicam import MultiRGBDCalibrated
from rdt.common import mc_util

In [43]:
mc_vis = meshcat.Visualizer(zmq_url='tcp://127.0.0.1:6001')
mc_vis['scene'].delete()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [3]:
from src.visualization.render_mp4 import mp4_from_pickle_jupyter
import os
os.environ['IMAGEIO_FFMPEG_EXE'] = '/usr/bin/ffmpeg'

base_path = Path("/home/anthony/repos/research/robust-rearrangement/src/real/test_refactor")
demo_path = base_path / "2024-05-02T19:45:35.pkl"
print(f'Base path: {base_path}\nDemo path: {demo_path}')

mp4_from_pickle_jupyter(demo_path, 'test.mp4')

Base path: /home/anthony/repos/research/robust-rearrangement/src/real/test_refactor
Demo path: /home/anthony/repos/research/robust-rearrangement/src/real/test_refactor/2024-05-02T19:45:35.pkl
File saved as test.mp4


In [12]:
base_path = Path("/home/anthony/repos/research/improbable_rdt/examples/teleop_demos/lamp_shade_pick_place_basic_rs2")
demo_path = base_path / "2025-05-01 17:44:36.703058" / "episode_data.pkl"
print(f'Base path: {base_path}\nDemo path: {demo_path}')

with open(demo_path, 'rb') as f:
    demo_data = pickle.load(f)

print(demo_data.keys())
for k in demo_data.keys():
    print(f'Key: {k}, Length: {len(demo_data[k])}, Type: {type(demo_data[k][0])}')

Base path: /home/anthony/repos/research/improbable_rdt/examples/teleop_demos/lamp_shade_pick_place_basic_rs2
Demo path: /home/anthony/repos/research/improbable_rdt/examples/teleop_demos/lamp_shade_pick_place_basic_rs2/2024-05-01 17:44:36.703058/episode_data.pkl
dict_keys(['robot_state', 'image_front', 'image_wrist', 'actions'])
Key: robot_state, Length: 119, Type: <class 'dict'>
Key: image_front, Length: 119, Type: <class 'dict'>
Key: image_wrist, Length: 119, Type: <class 'dict'>
Key: actions, Length: 119, Type: <class 'numpy.ndarray'>


In [15]:
print(demo_data['robot_state'][0].keys())
print(demo_data['image_front'][0].keys())
print(demo_data['image_wrist'][0].keys())

dict_keys(['ee_pose', 'joint_positions', 'gripper_width'])
dict_keys(['rgb', 'depth'])
dict_keys(['rgb', 'depth'])


In [35]:
depth_images = []
point_clouds = []
calib_fname = Path(path_util.get_rdt_src()) / "robot/camera_calibration_files" / "test/cam_0_calib_base_to_cam.json"
cam = MultiRGBDCalibrated(cam_names=['cam_0'], calib_filenames=[calib_fname]).cams[0]
# cam_intrinsics = np.array([
#     [613.14752197,   0.        , 326.19647217],
#     [  0.        , 613.16229248, 244.59855652],
#     [  0.        ,   0.        ,   1.        ]]
# )
cam_intrinsics = np.array([
       [385.75708008,   0.        , 326.95498657],
       [  0.        , 385.36810303, 237.9675293 ],
       [  0.        ,   0.        ,   1.        ]])

cam.cam_int_mat = cam_intrinsics
cam._init_pers_mat()
cam_pose_world = cam.cam_ext_mat


for i in range(n):
    rgb = demo_data['image_front'][i]['rgb']
    depth = demo_data['image_front'][i]['depth'] * 0.001
    valid = depth < cam.depth_max
    valid = np.logical_and(valid, depth > cam.depth_min)
    depth_valid = copy.deepcopy(depth)
    depth_valid[np.logical_not(valid)] = 0.0 # not exactly sure what to put for invalid depth

    pcd_cam = cam.get_pcd(in_world=False, filter_depth=False, rgb_image=rgb, depth_image=depth_valid)[0]
    pcd_cam_img = pcd_cam.reshape(depth.shape[0], depth.shape[1], 3)
    pcd_world = util.transform_pcd(pcd_cam, cam_pose_world)
    pcd_world_img = pcd_world.reshape(depth.shape[0], depth.shape[1], 3)
    pcd_dict = {
        'world': pcd_world,
        'cam': pcd_cam_img,
        'cam_img': pcd_cam,
        'world_img': pcd_world_img,
        'cam_pose_mat': cam_pose_world
    }

    depth_images.append(depth_valid)
    point_clouds.append(pcd_world)

Initializing camera cam_0


In [46]:
n = len(demo_data['robot_state'])

def state_to_ee_pose(rs):
    ee_pose_arr = rs['ee_pose']
    ee_pose_mat = np.eye(4)
    ee_pose_mat[:-1, -1] = ee_pose_arr[:3]
    ee_pose_mat[:-1, :-1] = R.from_quat(ee_pose_arr[3:7]).as_matrix()
    return ee_pose_mat


def action_to_des_ee_pose(act):
    des_ee_pose_arr = act[:-1] # no gripper
    des_ee_pose_mat = np.eye(4)
    des_ee_pose_mat[:-1, -1] = des_ee_pose_arr[:3]
    des_ee_pose_mat[:-1, :-1] = R.from_quat(des_ee_pose_arr[3:7]).as_matrix()
    return des_ee_pose_mat

idx = 0

mc_vis['scene/sa'].delete()

# plot the states
for i in range(n):
    ee_pose_mat = state_to_ee_pose(demo_data['robot_state'][i])
    des_ee_pose_mat = action_to_des_ee_pose(demo_data['actions'][i])

    mc_util.meshcat_frame_show(mc_vis, f'scene/sa/{idx}/{i}/state', ee_pose_mat)
    mc_util.meshcat_frame_show(mc_vis, f'scene/sa/{idx}/{i}/action', des_ee_pose_mat)

    idx = i // 10

In [47]:
mc_vis['scene/pcd'].delete()
pcd_idx = 21
mc_util.meshcat_pcd_show(mc_vis, point_clouds[pcd_idx], (0, 0, 0), name=f'scene/pcd/{pcd_idx}/point_cloud', size=0.002)

# mc_vis['scene/sa'].delete()
# for i in range(n):
#     mc_util.meshcat_pcd_show(mc_vis, point_clouds[i], (0, 0, 0), name=f'scene/pcd/point_cloud', size=0.002)
#     time.sleep(0.1)

#     ee_pose_mat = state_to_ee_pose(demo_data['robot_state'][i])
#     des_ee_pose_mat = action_to_des_ee_pose(demo_data['actions'][i])

#     mc_util.meshcat_frame_show(mc_vis, f'scene/sa/state', ee_pose_mat)
#     mc_util.meshcat_frame_show(mc_vis, f'scene/sa/action', des_ee_pose_mat)